# Examples of compartmental models
Using the nifty little functions we created in the file `utils.py`. 

This will look a lot like the Colab notebook we created.

In [ ]:
import utils

import matplotlib.pyplot as plt
import math 

In [ ]:
# With two populations: deer and grass model under Caughley's model, simulating for 30 years
# at discrete time step of 1 year.

# Define the starting populations in a dictionary
deer_grass_populations = {"deer": [10], "grass": [20]}

# Define some functions that are the components of the model

def deer_grows(state_dictionary, timestep):
    # Cauchley equation
    new_deer = state_dictionary["deer"] * 1.5 * (1 - math.exp(-0.001 * state_dictionary["grass"]))
    return {"deer": new_deer, "grass": 0}

def deer_dies(state_dictionary, timestep):
    # Deer declines 1.1 of population every year

    deer_change = -1.1 * state_dictionary["deer"]
    return {"deer": deer_change, "grass": 0}

def grass_grows(state_dictionary, timestep):
    # The grass grows logistically with a carrying capacity of 3000 and a rate of 0.8
    return {"deer": 0, "grass": 0.8 * state_dictionary["grass"] * (1 - state_dictionary["grass"] / 3000)}

def grass_dies(state_dictionary, timestep):
    # Cauchley equation
    return {"deer": 0, "grass": -1.2 * state_dictionary["deer"] * (1 - math.exp(-0.001 * state_dictionary["grass"]))}

# Compartments are on a new line just for readability
deer_grass_model = utils.CompartmentalModel(start_states=deer_grass_populations, 
                                                 compartments=[deer_grows, grass_grows, deer_dies, grass_dies],
                                                 time_units="years")

# Run the model for 14 years
deer_grass_model.simulate(timesteps=90)

# Check that it worked
deer_grass_model.print_states()


In [ ]:
# Plot

# Grab each set of states
all_states = deer_grass_model.get_states()
deer_states = all_states["deer"]
grass_states = all_states["grass"]

# Get the timestamps
timestamps = [i for i in range(len(deer_states))]

# Plot the deer and grass populations over time
# `scatter` plots points instead of a smooth line like `plot`
plt.step(timestamps, deer_states, label="Deer", color="tab:orange")
plt.step(timestamps, grass_states, label="Grass", color="tab:green")

# Add some labels
plt.title("Caughley Deer - Grass Model")
plt.xlabel(f"Time ({deer_grass_model.get_time_units()})")
plt.ylabel("Population")
plt.legend()

plt.show()
plt.close()